In [4]:
from langchain_google_genai import ChatGoogleGenerativeAI
import os
from pydantic import Field, BaseModel
from dotenv import load_dotenv
from langchain_core.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
load_dotenv()

True

In [2]:
model = ChatGoogleGenerativeAI(
    model='gemini-2.0-flash-lite',
    temperature=0,
    api_key = os.getenv('GEMINI_API_KEY')
)

In [5]:
class ResponseSchema(BaseModel):
    actual_input: str = Field(description="The actual user input")
    user_intent: str = Field(description="What the user have asked for that can be processed by LLM in next chain")

parser = PydanticOutputParser(pydantic_object=ResponseSchema)

In [9]:
prompt = PromptTemplate(
    template='''
You are an expert in Natural Language Understanding for Inventory Management Domain.

Right now you are assigned to understand what the user have asked and paraphrase it to send it to the next LLM agent.
So you understand what the user have asked in prompt and return the same information, but in the format where agent can understand better.

The user input: {text}

The output should be in the following format:
{response_schema}
''',
input_variables=['text'],
partial_variables={'response_schema': parser.get_format_instructions()}
)

In [10]:
chain = prompt | model | parser

In [11]:
response = chain.invoke(input={"text": "Do we have any adhesives available for rubber tyres? If yes, where is it kept in the inventory?"})

print(response)

actual_input='Do we have any adhesives available for rubber tyres? If yes, where is it kept in the inventory?' user_intent='Check inventory for adhesives suitable for rubber tires and identify their storage location.'


In [12]:
for key, val in response:
    print(f"{key}: {val}")

actual_input: Do we have any adhesives available for rubber tyres? If yes, where is it kept in the inventory?
user_intent: Check inventory for adhesives suitable for rubber tires and identify their storage location.
